# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope(y_axis):
    global data
    ma_len = len(y_axis)
    
    x_axis = []
    for i in range(ma_len):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    tick_act = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_act.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
    temp_df['tick_act'] = tick_act      
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_ma(ma_list):
    global data
    ma_len = len(ma_list)
    sema_val = list(pd.DataFrame(ma_list).ewm(span=ma_len).mean()[0])[ma_len - 1]    
    return(sema_val)

## File paths

#### 1. High number of ticks ensures pip level movement and pip level predictions
#### 2. High sma_len helps ml model prediction accuracy (Predictable curve)
#### 3. High sma_len remove connection between actual tick_avg and sema (Prediction is high, but actual prediction is poor)

In [6]:
year = 2019

data = {}
data['number_of_ticks'] = 300
data['rsi_window'] = 14
data['sma_len'] = 5
data['lma_len'] = 10

diff_col = 'sema'
#diff_col = 'tick_avg'

data['pip_diff'] = 0.0001

source_file_path = f'data/yearly_tick_data/{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/yearly_tick_data/2019.csv
chunk_file_path : data/yearly_tick_data\chunk_2019.csv
target_file_path : data/yearly_tick_data\tab_2019.csv


## Read data

In [7]:
%%time
#df = pd.read_csv(source_file_path, nrows=100000)
df = pd.read_csv(source_file_path)
df.head()

Wall time: 19.1 s


,DateTime,Bid,Ask,Volume
0,20190101 22:02:37.254,1.14598,1.14682,4
1,20190101 22:02:38.590,1.14599,1.14682,2
2,20190101 22:02:39.138,1.14599,1.14684,4
3,20190101 22:02:55.787,1.14598,1.14684,4
4,20190101 22:03:02.060,1.14598,1.14684,4


## Data manipulation

In [8]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|█████████████████████████████████████████████| 97288/97288 [00:30<00:00, 3179.74it/s]


Records : 97288
Wall time: 33.1 s


,tick_act,tick_avg,spread_avg,tick_sd
0,1.146260,1.146473,0.000481,0.000160
1,1.146320,1.146313,0.000478,0.000066
2,1.146325,1.146300,0.000327,0.000062
3,1.146315,1.146311,0.000254,0.000030
4,1.146530,1.146350,0.000418,0.000063


In [9]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 73.1 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_ma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_ma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

97284it [01:33, 1035.33it/s]
97279it [01:39, 973.95it/s] 

Wall time: 3min 13s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope)

97280it [00:31, 3042.80it/s]
97275it [00:31, 3051.06it/s]

Wall time: 1min 3s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
del df['tick_act']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 97275
Wall time: 118 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
97270,1.121664,0.000032,0.000059,1.121650,1.121737,-0.000002,-0.000039,0.000068,0.000010,0.000078,...,-0.000065,-0.000170,1.121737,1.121596,0.000073,-0.000068,-78.436454,-82.698052,-0.000088,same
97271,1.121606,0.000038,0.000107,1.121629,1.121697,-0.000021,-0.000041,-0.000058,0.000010,0.000071,...,-0.000060,-0.000136,1.121664,1.121596,0.000058,-0.000010,-72.349933,-81.597907,-0.000068,same
97272,1.121467,0.000034,0.000033,1.121566,1.121639,-0.000063,-0.000058,-0.000139,0.000010,0.000077,...,-0.000057,-0.000115,1.121664,1.121467,0.000197,0.000000,-71.941568,-80.551140,-0.000073,same
97273,1.121420,0.000050,0.000056,1.121505,1.121582,-0.000061,-0.000057,-0.000047,0.000006,0.000081,...,-0.000059,-0.000104,1.121664,1.121420,0.000244,0.000000,-75.145616,-79.438822,-0.000077,decrease
97274,1.121250,0.000101,0.000088,1.121403,1.121502,-0.000102,-0.000081,-0.000169,0.000006,0.000082,...,-0.000072,-0.000101,1.121664,1.121250,0.000413,0.000000,-80.810470,-78.741640,-0.000099,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 3.94 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       76936   79.091236
decrease   10432   10.724235
increase    9907   10.184528


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.146674,0.000066,0.000033,1.146645,1.146574,0.000022,0.000035,0.000021,0.000045,0.000030,...,0.000036,0.000133,1.146674,1.146587,0.000000,-0.000087,72.840045,78.091169,0.000071,same
1,1.146695,0.000064,0.000047,1.146665,1.146606,0.000020,0.000032,0.000021,0.000046,0.000030,...,0.000035,0.000112,1.146695,1.146587,0.000000,-0.000108,68.684905,78.874888,0.000059,same
2,1.146682,0.000056,0.000017,1.146675,1.146635,0.000011,0.000029,-0.000013,0.000046,0.000020,...,0.000054,0.000077,1.146695,1.146613,0.000013,-0.000069,63.352971,77.576158,0.000041,same
3,1.146594,0.000053,0.000024,1.146648,1.146637,-0.000028,0.000002,-0.000088,0.000046,0.000025,...,0.000034,0.000039,1.146695,1.146594,0.000101,0.000000,38.347811,73.138665,0.000010,same
4,1.146523,0.000050,0.000051,1.146599,1.146617,-0.000048,-0.000020,-0.000071,0.000045,0.000030,...,0.000002,0.000011,1.146695,1.146523,0.000172,0.000000,-47.239966,62.713918,-0.000018,decrease


In [15]:
# Direction -------------------------
#diff_col = 'tick_act'
#data['pip_diff'] = 0.0001
diff_col = 'tick_avg'

df['act_direction'] = 'same'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['act_direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

print('prediction : same')
print(df.loc[df['direction'] == 'same', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : increase')
print(df.loc[df['direction'] == 'increase', 'act_direction'].value_counts(normalize=True))
print('-------------')

print('prediction : decrease')
print(df.loc[df['direction'] == 'decrease', 'act_direction'].value_counts(normalize=True))
print('-------------')

df['tick_avg_diff'] = round(df['tick_avg'].diff() * 10000)

print('\n')
print(df[['tick_avg_diff','tick_avg', 'sema', 'direction','act_direction']].head(50))
del df['act_direction']
del df['tick_avg_diff']

prediction : same
same        0.642625
increase    0.181670
decrease    0.175704
Name: act_direction, dtype: float64
-------------
prediction : increase
increase    0.789038
same        0.196124
decrease    0.014838
Name: act_direction, dtype: float64
-------------
prediction : decrease
decrease    0.788344
same        0.196223
increase    0.015433
Name: act_direction, dtype: float64
-------------


    tick_avg_diff  tick_avg      sema direction act_direction
0             NaN  1.146674  1.146645      same          same
1             0.0  1.146695  1.146665      same          same
2            -0.0  1.146682  1.146675      same          same
3            -1.0  1.146594  1.146648      same          same
4            -1.0  1.146523  1.146599  decrease      decrease
5            -3.0  1.146239  1.146457  decrease          same
6            -1.0  1.146155  1.146329  decrease      decrease
7            -2.0  1.145950  1.146166      same      increase
8             1.0  1.146061  1.146104  